#  2020AIML544 - TEXT MINING - ASSIGNMENT - 1

### TASKS PERFORMED:
    1. IMPORT REQUIRED LIBRARIES
    2. READ THE TEXT DATA FROM EXCEL FILE INTO A DATAFRAME AND  
       PERFORM PRE-PROCESSING
       i.  REMOVE SPECIAL CHARACTERS FROM THE TEXT DATA
       ii. CONVERT THE TEXT TO LOWER CASE
       iii.REMOVE URLS IF PRESENT IN THE DATA
       iv. REMOVE STOP WORDS IN THE TEXT DATA
    3. TRAIN TEST SPLIT OF DATASET
    4. CREATE PIPELINE AND DEFINE PARAMETERS FOR GRIDSEARCH
    5. PERFORM CLASSIFICATION USING GRIDSEARCH
    6. PRINT CONFUSION MATRIX AND CLASSIFICATION REPORT

### <span style='background : yellow' > **TASK 1:**</span> IMPORT REQUIRED LIBRARIES

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

from pandas import DataFrame
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV 

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

### <span style='background : yellow' > **TASK 2:**</span> READ DATASET AND PERFORM PREPROCESSING 

#### Read the input excel file into a dataframe

In [2]:
file_path = r"C:\Users\Aspire V3-572\Corona_NLP_train.CSV"
df = pd.read_csv(file_path, header=0, encoding = "ISO-8859-1")

#### Display information about the dataframe using head function

In [3]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order,Positive
2,3801,48753,Vagabonds,16-03-2020,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive
3,3802,48754,NaN,16-03-2020,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j",Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don't panic. It causes shortage...\r\r\n\r\r\n#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n",Extremely Negative


In [4]:
#We just need 2 columns for this assignment hence remove the others from the dataframe
df = df[['OriginalTweet','Sentiment']]
df.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,Neutral
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order,Positive
2,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive
3,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j",Positive
4,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don't panic. It causes shortage...\r\r\n\r\r\n#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n",Extremely Negative


In [5]:
df.shape

(41157, 2)

There are 41157 records in the dataset.

#### REMOVE SPECIAL CHARACTERS FROM THE TEXT DATA

In [6]:
df1 = df.replace(r'[^A-Za-z0-9\n ]+', '', regex=True)
df1.head()

,OriginalTweet,Sentiment
0,MeNyrbie PhilGahan Chrisitv httpstcoiFz9FAn2Pa and httpstcoxX6ghGFzCC and httpstcoI2NlzdxNo8,Neutral
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order,Positive
2,Coronavirus Australia Woolworths to give elderly disabled dedicated shopping hours amid COVID19 outbreak httpstcobInCA9Vp8P,Positive
3,My food stock is not the only one which is empty\n\nPLEASE dont panic THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need \nStay calm stay safe\n\nCOVID19france COVID19 COVID19 coronavirus confinement Confinementotal ConfinementGeneral httpstcozrlG0Z520j,Positive
4,Me ready to go at supermarket during the COVID19 outbreak\n\nNot because Im paranoid but because my food stock is litteraly empty The coronavirus is a serious thing but please dont panic It causes shortage\n\nCoronavirusFrance restezchezvous StayAtHome confinement httpstcousmuaLq72n,Extremely Negative


#### CONVERT TEXT TO LOWER CASE

In [7]:
df2 = df1.applymap(lambda s:s.lower() if type(s) == str else s)
df2.head()

,OriginalTweet,Sentiment
0,menyrbie philgahan chrisitv httpstcoifz9fan2pa and httpstcoxx6ghgfzcc and httpstcoi2nlzdxno8,neutral
1,advice talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist gp set up online shopping accounts if poss adequate supplies of regular meds but not over order,positive
2,coronavirus australia woolworths to give elderly disabled dedicated shopping hours amid covid19 outbreak httpstcobinca9vp8p,positive
3,my food stock is not the only one which is empty\n\nplease dont panic there will be enough food for everyone if you do not take more than you need \nstay calm stay safe\n\ncovid19france covid19 covid19 coronavirus confinement confinementotal confinementgeneral httpstcozrlg0z520j,positive
4,me ready to go at supermarket during the covid19 outbreak\n\nnot because im paranoid but because my food stock is litteraly empty the coronavirus is a serious thing but please dont panic it causes shortage\n\ncoronavirusfrance restezchezvous stayathome confinement httpstcousmualq72n,extremely negative


#### REMOVE WEBSITE URLs FROM THE TEXT DATA

In [8]:
df2['OriginalTweet'] = df2['OriginalTweet'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
df3 = df2.copy()
df3.head()

,OriginalTweet,Sentiment
0,menyrbie philgahan chrisitv and and,neutral
1,advice talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist gp set up online shopping accounts if poss adequate supplies of regular meds but not over order,positive
2,coronavirus australia woolworths to give elderly disabled dedicated shopping hours amid covid19 outbreak,positive
3,my food stock is not the only one which is empty\n\nplease dont panic there will be enough food for everyone if you do not take more than you need \nstay calm stay safe\n\ncovid19france covid19 covid19 coronavirus confinement confinementotal confinementgeneral,positive
4,me ready to go at supermarket during the covid19 outbreak\n\nnot because im paranoid but because my food stock is litteraly empty the coronavirus is a serious thing but please dont panic it causes shortage\n\ncoronavirusfrance restezchezvous stayathome confinement,extremely negative


#### REMOVE STOP WORDS FROM THE TEXT DATA

In [9]:
stop_words = stopwords.words('english')
df3['OriginalTweet'] = df3['OriginalTweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df3.head()

,OriginalTweet,Sentiment
0,menyrbie philgahan chrisitv,neutral
1,advice talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist gp set online shopping accounts poss adequate supplies regular meds order,positive
2,coronavirus australia woolworths give elderly disabled dedicated shopping hours amid covid19 outbreak,positive
3,food stock one empty please dont panic enough food everyone take need stay calm stay safe covid19france covid19 covid19 coronavirus confinement confinementotal confinementgeneral,positive
4,ready go supermarket covid19 outbreak im paranoid food stock litteraly empty coronavirus serious thing please dont panic causes shortage coronavirusfrance restezchezvous stayathome confinement,extremely negative


### <span style='background : yellow' > **TASK 3:**</span> SPLIT TRAIN & TEST DATASET USING SKLEARN

In [10]:
X = df3['OriginalTweet']
y = df3['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0,stratify=y) 

### <span style='background : yellow' > **TASK 4:**</span> CREATE PIPELINE AND DEFINE PARAMETERS FOR GRIDSEARCH

In [17]:
pipeline = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2', 'None'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

### <span style='background : yellow' > **TASK 5:**</span> PERFORM CLASSIFICATION (USING GRIDSEARCH)

In [18]:
gs_clf = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1, verbose=1, scoring='accuracy')

gs_clf.fit(X_train, y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  6.3min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [1, 0.1, 0.01],
                         'tfidf__norm': ('l1', 'l2', 'None'),
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2), (2, 2)]},
             scoring='accuracy', verbose=1)

In [19]:
# PRINT THE BEST SCORE AND PARAMETERS

print('best score:', gs_clf.best_score_)

for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

best score: 0.442291989469943
clf__alpha: 0.1
tfidf__norm: 'l2'
tfidf__use_idf: False
vect__ngram_range: (1, 1)


### <span style='background : yellow' > **TASK 6:**</span> PRINT CONFUSION MATRIX & CLASSIFICATION REPORT

In [20]:
y_pred = gs_clf.predict(X_test)

pd.crosstab(y_test,y_pred,rownames=['Predicted'],colnames=['True'],margins=True)

True,extremely negative,extremely positive,negative,neutral,positive,All
Predicted,,,,,,
extremely negative,339,10,989,39,268,1645
extremely positive,6,458,132,25,1366,1987
negative,120,39,1589,136,1091,2975
neutral,14,40,463,745,1052,2314
positive,22,178,634,168,2425,3427
All,501,725,3807,1113,6202,12348


In [21]:
print('Accuracy :', accuracy_score(y_test, y_pred))
print('f1 score :', f1_score(y_test, y_pred, average='weighted'))

print('\n\nCLASSIFICATION REPORT \n\n', classification_report(y_test, y_pred))

Accuracy : 0.4499514091350826
f1 score : 0.430606401151401


CLASSIFICATION REPORT 

                     precision    recall  f1-score   support

extremely negative       0.68      0.21      0.32      1645
extremely positive       0.63      0.23      0.34      1987
          negative       0.42      0.53      0.47      2975
           neutral       0.67      0.32      0.43      2314
          positive       0.39      0.71      0.50      3427

          accuracy                           0.45     12348
         macro avg       0.56      0.40      0.41     12348
      weighted avg       0.53      0.45      0.43     12348

